In [1]:
#| include: false
using Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
cd(@__DIR__)

  Activating project at `~/gitrepos/kdheepak.github.io/blog/effect-of-type-inference-on-performance-in-julia`


In [2]:
abstract type Shape end
area(::Shape) = 0.0

struct Square <: Shape
    side::Float64
end
area(s::Square) = s.side * s.side
    
struct Rectangle <: Shape
    width::Float64
    height::Float64
end
area(r::Rectangle) = r.width * r.height
    
struct Triangle <: Shape
    base::Float64
    height::Float64
end
area(t::Triangle) = 1.0/2.0 * t.base * t.height

struct Circle <: Shape
    radius::Float64
end
area(c::Circle) = π * c.radius^2

area (generic function with 5 methods)

In [3]:
using Test
@testset "Areas" begin
    @test area(Square(2)) == 4
    @test area(Rectangle(2,3)) == 6
    @test area(Triangle(2,3)) == 3
    @test area(Circle(2)) ≈ 4π
end;

Test Summary: | Pass  Total  Time
Areas         |    4      4  0.2s


In [4]:
using Random

Random.seed!(42)

function shape_builder(choice::Integer)
    if choice == 1
        Square(rand())
    elseif choice == 2
        Rectangle(rand(), rand())
    elseif choice == 3
        Triangle(rand(), rand())
    elseif choice == 4
        Circle(rand())
    end
end

count = 1_000_000
shapes = [shape_builder(rand((1,2,3,4))) for _ in 1:count];

In [5]:
#| echo: false
using Format
using Markdown
l = cfmt("%'d", length(shapes))
Markdown.md"The total number of shapes we have is $l."

The total number of shapes we have is 1,000,000.


In [6]:
typeof(shapes)

Vector{Shape} (alias for Array{Shape, 1})

In [7]:
main1(shapes) = sum(area.(shapes))

main1 (generic function with 1 method)

In [8]:
@time main1(shapes);

  0.088708 seconds (2.11 M allocations: 45.631 MiB, 11.45% gc time, 34.32% compilation time)


In [9]:
using BenchmarkTools

In [10]:
@benchmark main1(shapes)

BenchmarkTools.Trial: 113 samples with 1 evaluation.
 Range (min … max):  43.498 ms …  46.192 ms  ┊ GC (min … max): 0.80% … 0.85%
 Time  (median):     44.254 ms               ┊ GC (median):    0.82%
 Time  (mean ± σ):   44.346 ms ± 506.041 μs  ┊ GC (mean ± σ):  0.83% ± 0.45%

            ▃▃█ ▃ ▃▅ ▃  ▂                                       
  ▅▅▇▁▁▄▁▄█▁███▅█████████▅▇▅▄▇▅▅▄▄▁▄▇▁▁▁▁▄▁▁▄▅▁▄▄▄▁▁▁▁▁▁▁▄▁▁▁▄ ▄
  43.5 ms         Histogram: frequency by time           46 ms <

 Memory estimate: 38.14 MiB, allocs estimate: 1999492.

In [11]:
shapes_by_type(::Type{T}, shapes) where T = [shape for shape in shapes if isa(shape, T)]

square_arr = shapes_by_type(Square, shapes)
rectangle_arr = shapes_by_type(Rectangle, shapes)
triangle_arr = shapes_by_type(Triangle, shapes)
circle_arr = shapes_by_type(Circle, shapes)

nothing

In [12]:
typeof(square_arr)

Vector{Square} (alias for Array{Square, 1})

In [13]:
typeof(rectangle_arr)

Vector{Rectangle} (alias for Array{Rectangle, 1})

In [14]:
typeof(triangle_arr)

Vector{Triangle} (alias for Array{Triangle, 1})

In [15]:
typeof(circle_arr)

Vector{Circle} (alias for Array{Circle, 1})

In [43]:
filter_shapes_by_type(::Type{T}, shapes) where T = filter(s -> isa(s, T), shapes)

shape_arr1 = filter_shapes_by_type(Square, shapes)
shape_arr2 = filter_shapes_by_type(Rectangle, shapes)
shape_arr3 = filter_shapes_by_type(Triangle, shapes)
shape_arr4 = filter_shapes_by_type(Circle, shapes)

nothing

In [44]:
typeof(shape_arr1)

Vector{Shape} (alias for Array{Shape, 1})

In [45]:
typeof(shape_arr2)

Vector{Shape} (alias for Array{Shape, 1})

In [46]:
typeof(shape_arr3)

Vector{Shape} (alias for Array{Shape, 1})

In [47]:
typeof(shape_arr4)

Vector{Shape} (alias for Array{Shape, 1})

In [48]:
sorted_shapes_shape = vcat(square_arr, rectangle_arr, triangle_arr, circle_arr);
sorted_shapes_any = Any[s for s in sorted_shapes_shape];

In [49]:
typeof(sorted_shapes_shape)

Vector{Shape} (alias for Array{Shape, 1})

In [50]:
@benchmark main1(sorted_shapes_shape)

BenchmarkTools.Trial: 139 samples with 1 evaluation.
 Range (min … max):  33.861 ms … 40.148 ms  ┊ GC (min … max): 0.00% … 13.83%
 Time  (median):     34.456 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   36.007 ms ±  2.299 ms  ┊ GC (mean ± σ):  4.60% ±  5.54%

   ▇▄ █                                                        
  ▄██▇█▆█▅▄▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▃▃▃▅▆▆▆▆▆▆▃▃▃▄ ▃
  33.9 ms         Histogram: frequency by time        39.6 ms <

 Memory estimate: 38.14 MiB, allocs estimate: 1999492.

In [51]:
typeof(sorted_shapes_any)

Vector{Any} (alias for Array{Any, 1})

In [52]:
@benchmark main1(sorted_shapes_any)

BenchmarkTools.Trial: 340 samples with 1 evaluation.
 Range (min … max):  13.443 ms … 20.388 ms  ┊ GC (min … max): 0.00% … 29.57%
 Time  (median):     13.767 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   14.730 ms ±  1.847 ms  ┊ GC (mean ± σ):  6.59% ± 10.09%

  ▂█▂                                                          
  ████▅▄▃▃▃▃▃▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▂▃▂▃▃▃▃▃▂▃▃▂▃▂▃▂▃▂▁▃▂ ▃
  13.4 ms         Histogram: frequency by time        19.3 ms <

 Memory estimate: 22.89 MiB, allocs estimate: 1000010.

In [53]:
main2(arrs...) = sum(main1.(arrs))

main2 (generic function with 1 method)

In [54]:
@time main2(shape_arr1, shape_arr2, shape_arr3, shape_arr4);

  0.044200 seconds (2.00 M allocations: 38.185 MiB, 11.74% gc time, 7.98% compilation time)


In [55]:
@benchmark main2(shape_arr1, shape_arr2, shape_arr3, shape_arr4)

BenchmarkTools.Trial: 138 samples with 1 evaluation.
 Range (min … max):  34.069 ms … 44.136 ms  ┊ GC (min … max): 0.00% … 19.20%
 Time  (median):     34.815 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   36.328 ms ±  2.249 ms  ┊ GC (mean ± σ):  4.59% ±  5.44%

    █▆▅                                                        
  ▃▇███▆▅▃▁▃▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆▇▇▅▄▃▃▇▃▃▃▁▅▃▃▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▃ ▃
  34.1 ms         Histogram: frequency by time        42.5 ms <

 Memory estimate: 38.12 MiB, allocs estimate: 1997974.

In [56]:
@time main2(square_arr, rectangle_arr, triangle_arr, circle_arr);

  0.005050 seconds (953 allocations: 7.678 MiB, 76.24% compilation time)


In [57]:
@benchmark main2(square_arr, rectangle_arr, triangle_arr, circle_arr)

BenchmarkTools.Trial: 4267 samples with 1 evaluation.
 Range (min … max):  491.542 μs … 11.094 ms  ┊ GC (min … max):  0.00% … 89.12%
 Time  (median):     884.583 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):     1.167 ms ±  1.023 ms  ┊ GC (mean ± σ):  23.41% ± 21.20%

   ▁▃▇█▆▄▃▁                                                    ▁
  ▇████████▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▃▅▆▆▇▇▇▇█▇█▇▇▇▆▇▆▇▆▅▅▅▆▄▅▅▅▃▅▅▃▄▅▃ █
  492 μs        Histogram: log(frequency) by time       5.8 ms <

 Memory estimate: 7.63 MiB, allocs estimate: 20.